# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096915


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   9%|▉         | 3/32 [00:03<00:29,  1.02s/it]

Rendering models:  12%|█▎        | 4/32 [00:03<00:22,  1.24it/s]

Rendering models:  19%|█▉        | 6/32 [00:06<00:25,  1.01it/s]

Rendering models:  22%|██▏       | 7/32 [00:06<00:19,  1.31it/s]

...

Rendering models:  91%|█████████ | 29/32 [00:14<00:00,  3.64it/s]

Rendering models:  94%|█████████▍| 30/32 [00:14<00:00,  3.50it/s]

Rendering models:  97%|█████████▋| 31/32 [00:14<00:00,  3.30it/s]

Rendering models: 100%|██████████| 32/32 [00:15<00:00,  2.81it/s]

martianmax                            0.000200
ethanth                               0.000307
Idax98                                0.012761
Linda_J._Berkel                       0.000135
Jaronec                               0.021559
ellivc                                0.000610
not-logged-in-c6917d9b19f41e6c2995    0.006694
awright5                              0.002159
Galaxy-one                            0.000220
CScampstudent                         0.031603
sn346808                              0.024317
ga05july                              0.000465
not-logged-in-14d09f70263c1fe15cc3    0.001592
KeatonS                               0.001877
gadam8                                0.000160
Myriampe                              0.002377
not-logged-in-9dfe008ef2c8408f69b2    0.000250
umtuck1                               0.041099
umtuck1                               0.000429
umtuck1                               0.000624
ranyel.a.e.pasimio                    0.000968
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())